In [1]:
import pandas as pd

In [5]:
import pandas as pd
import os

# CSV 파일들의 경로를 리스트로 저장
csv_files = [f'../processing_data/products_with_money{i}.csv' for i in range(1, 19)]

# 빈 리스트를 만들어 데이터를 저장
data_frames = []

# 각 CSV 파일을 읽어와 리스트에 추가
for file in csv_files:
    try:
        df = pd.read_csv(file, encoding='euc-kr')
        data_frames.append(df)
    except UnicodeDecodeError:
        try:
            df = pd.read_csv(file, encoding='utf-8')
            data_frames.append(df)
        except UnicodeDecodeError:
            try:
                df = pd.read_csv(file, encoding='cp949')
                data_frames.append(df)
            except UnicodeDecodeError:
                print(f"파일 {file}을(를) 읽는 중 오류가 발생했습니다.")

# 모든 데이터를 하나의 DataFrame으로 합치기
if data_frames:
    merged_df = pd.concat(data_frames, ignore_index=True)

    # 합쳐진 DataFrame 확인
    print(merged_df.head())

    # 필요시 합쳐진 데이터를 새로운 CSV 파일로 저장
    merged_df.to_csv('../processing_data/products_price.csv', index=False)
else:
    print("합칠 데이터가 없습니다.")


   product_id                                       product_name  aisle_id  \
0           1                         Chocolate Sandwich Cookies        61   
1           2                                   All-Seasons Salt       104   
2           3               Robust Golden Unsweetened Oolong Tea        94   
3           4  Smart Ones Classic Favorites Mini Rigatoni Wit...        38   
4           5                          Green Chile Anytime Sauce         5   

   department_id  money  
0           19.0  $6.99  
1           13.0  $5.99  
2            7.0  $3.39  
3            1.0  $3.59  
4           13.0  $8.99  


In [15]:
제품 =  pd.read_csv('../data/제품.csv', encoding='cp949')
제품

,제품ID,제품이름,소분류,대분류
0,1,Chocolate Sandwich Cookies,cookies cakes,snacks
1,2,All-Seasons Salt,spices seasonings,pantry
2,3,Robust Golden Unsweetened Oolong Tea,tea,beverages
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,frozen meals,frozen
4,5,Green Chile Anytime Sauce,marinades meat preparation,pantry
...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",spirits,alcohol
49684,49685,En Croute Roast Hazelnut Cranberry,frozen vegan vegetarian,frozen
49685,49686,Artisan Baguette,bread,bakery
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,cat food care,pets


In [23]:
제품.isna().sum()

제품ID    0
제품이름    0
소분류     0
대분류     0
dtype: int64

In [20]:
제품_가격 = pd.read_csv('../processing_data/products_price.csv', encoding='cp949')
제품_가격 

,product_id,product_name,aisle_id,department_id,money
0,1,Chocolate Sandwich Cookies,61,19.0,$6.99
1,2,All-Seasons Salt,104,13.0,$5.99
2,3,Robust Golden Unsweetened Oolong Tea,94,7.0,$3.39
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1.0,$3.59
4,5,Green Chile Anytime Sauce,5,13.0,$8.99
...,...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5.0,$22.69
49684,49685,En Croute Roast Hazelnut Cranberry,42,1.0,$12.00
49685,49686,Artisan Baguette,112,3.0,$2.99
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8.0,$1.49


In [21]:
# 'money' 열에서 '$' 제거하고 float 타입으로 변환
제품_가격['money'] = 제품_가격['money'].str.split(' ').str[0].str.replace('$', '').astype(float)

# 결과 확인
제품_가격

,product_id,product_name,aisle_id,department_id,money
0,1,Chocolate Sandwich Cookies,61,19.0,6.99
1,2,All-Seasons Salt,104,13.0,5.99
2,3,Robust Golden Unsweetened Oolong Tea,94,7.0,3.39
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1.0,3.59
4,5,Green Chile Anytime Sauce,5,13.0,8.99
...,...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5.0,22.69
49684,49685,En Croute Roast Hazelnut Cranberry,42,1.0,12.00
49685,49686,Artisan Baguette,112,3.0,2.99
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8.0,1.49


In [22]:
제품_가격.isna().sum()

product_id          0
product_name        0
aisle_id            0
department_id       1
money            5544
dtype: int64

In [24]:
# 'money'의 결측치를 'aisle_id'별 'money'의 평균으로 채우기
제품_가격['money'] = 제품_가격.groupby('aisle_id')['money'].transform(lambda x: x.fillna(x.mean()))

# 결과 확인
print(제품_가격.head())
print(제품_가격.isna().sum())

   product_id                                       product_name  aisle_id  \
0           1                         Chocolate Sandwich Cookies        61   
1           2                                   All-Seasons Salt       104   
2           3               Robust Golden Unsweetened Oolong Tea        94   
3           4  Smart Ones Classic Favorites Mini Rigatoni Wit...        38   
4           5                          Green Chile Anytime Sauce         5   

   department_id  money  
0           19.0   6.99  
1           13.0   5.99  
2            7.0   3.39  
3            1.0   3.59  
4           13.0   8.99  
product_id       0
product_name     0
aisle_id         0
department_id    1
money            0
dtype: int64


In [28]:
# '제품ID'와 'product_id'를 기준으로 병합
merged_df = pd.merge(제품, 제품_가격[['product_id', 'money']], left_on='제품ID', right_on='product_id', how='left')

# 필요없는 'product_id' 열 삭제
merged_df.drop(columns=['product_id'], inplace=True)

# 'money' 열을 '가격'으로 이름 변경
merged_df.rename(columns={'money': '가격'}, inplace=True)

# 결과 확인
merged_df.head()

# 필요시 합쳐진 데이터를 새로운 CSV 파일로 저장
merged_df.to_csv('../data/제품.csv', index=False, encoding='euc-kr')

In [41]:
제품 =  pd.read_csv('../data/제품.csv', encoding='euc-kr')
제품

,제품ID,제품이름,소분류,대분류,가격
0,1,Chocolate Sandwich Cookies,cookies cakes,snacks,6.99
1,2,All-Seasons Salt,spices seasonings,pantry,5.99
2,3,Robust Golden Unsweetened Oolong Tea,tea,beverages,3.39
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,frozen meals,frozen,3.59
4,5,Green Chile Anytime Sauce,marinades meat preparation,pantry,8.99
...,...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",spirits,alcohol,22.69
49684,49685,En Croute Roast Hazelnut Cranberry,frozen vegan vegetarian,frozen,12.00
49685,49686,Artisan Baguette,bread,bakery,2.99
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,cat food care,pets,1.49


In [42]:
주문_제품 = pd.read_csv('../data/주문_제품.csv').drop(columns=['Unnamed: 0'])
주문_제품

,주문ID,제품ID,장바구니순서,재주문,제품이름,소분류,대분류
0,2,33120,1,1,Organic Egg Whites,eggs,dairy eggs
1,2,28985,2,1,Michigan Organic Kale,fresh vegetables,produce
2,2,9327,3,0,Garlic Powder,spices seasonings,pantry
3,2,45918,4,1,Coconut Butter,oils vinegars,pantry
4,2,30035,5,0,Natural Sweetener,baking ingredients,pantry
...,...,...,...,...,...,...,...
33819101,3421063,14233,3,1,Natural Artesian Water,water seltzer sparkling water,beverages
33819102,3421063,35548,4,1,Twice Baked Potatoes,prepared meals,deli
33819103,3421070,35951,1,1,Organic Unsweetened Almond Milk,soy lactosefree,dairy eggs
33819104,3421070,16953,2,1,Creamy Peanut Butter,spreads,pantry


In [43]:
# 제품ID를 기준으로 병합
주문_제품 = pd.merge(주문_제품, 제품[['제품ID', '가격']], left_on='제품ID', right_on='제품ID', how='left')
주문_제품

,주문ID,제품ID,장바구니순서,재주문,제품이름,소분류,대분류,가격
0,2,33120,1,1,Organic Egg Whites,eggs,dairy eggs,8.39
1,2,28985,2,1,Michigan Organic Kale,fresh vegetables,produce,2.79
2,2,9327,3,0,Garlic Powder,spices seasonings,pantry,5.99
3,2,45918,4,1,Coconut Butter,oils vinegars,pantry,11.99
4,2,30035,5,0,Natural Sweetener,baking ingredients,pantry,7.99
...,...,...,...,...,...,...,...,...
33819101,3421063,14233,3,1,Natural Artesian Water,water seltzer sparkling water,beverages,3.99
33819102,3421063,35548,4,1,Twice Baked Potatoes,prepared meals,deli,1.28
33819103,3421070,35951,1,1,Organic Unsweetened Almond Milk,soy lactosefree,dairy eggs,6.49
33819104,3421070,16953,2,1,Creamy Peanut Butter,spreads,pantry,4.69


In [49]:
# 주문ID 별 가격의 합계를 계산
주문_가격 = 주문_제품.groupby('주문ID')['가격'].sum().reset_index()

# 결과 확인
주문_가격

,주문ID,가격
0,1,58.43
1,2,58.41
2,3,45.12
3,4,136.00
4,5,193.43
...,...,...
3346078,3421079,14.00
3346079,3421080,59.79
3346080,3421081,35.05
3346081,3421082,58.73


In [46]:
# 필요시 합쳐진 데이터를 새로운 CSV 파일로 저장
주문_제품.to_csv('../data/주문_제품.csv', index=False, encoding='utf-8')

In [48]:
주문_고객 = pd.read_csv('../data/주문_고객.csv').drop(columns=['Unnamed: 0'])
주문_고객

,주문ID,고객ID,데이터유형,주문생성횟수,주문생성요일,주문생성시간,경과일수,제품수,냉동식품,기타,...,"육류,해산물",식료품,아침식사,통조림,"유제품,계란",가정용품,아기용품,간식,델리,누락
0,2539329,1,prior,1,화요일,8,NaN,5.0,0,0,...,0,0,0,0,1,1,0,2,0,0
1,2398795,1,prior,2,수요일,7,15.0,6.0,0,0,...,0,0,1,0,0,0,0,3,0,0
2,473747,1,prior,3,수요일,12,21.0,5.0,0,0,...,0,1,0,0,1,0,0,2,0,0
3,2254736,1,prior,4,목요일,7,29.0,5.0,0,0,...,0,0,0,0,1,1,0,2,0,0
4,431534,1,prior,5,목요일,15,28.0,8.0,0,0,...,0,0,0,0,1,0,0,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3346078,2266710,206209,prior,10,금요일,18,29.0,9.0,1,0,...,0,1,1,0,1,1,0,1,0,0
3346079,1854736,206209,prior,11,목요일,10,30.0,8.0,0,0,...,0,0,0,0,2,1,0,1,1,0
3346080,626363,206209,prior,12,월요일,12,18.0,20.0,0,0,...,0,1,2,0,4,3,0,1,1,0
3346081,2977660,206209,prior,13,월요일,12,7.0,9.0,0,0,...,0,0,1,0,2,0,0,4,0,0


In [50]:
# 주문_고객과 주문_가격 데이터프레임을 주문ID를 기준으로 병합
주문_고객 = pd.merge(주문_고객, 주문_가격, on='주문ID', how='left')
주문_고객

,주문ID,고객ID,데이터유형,주문생성횟수,주문생성요일,주문생성시간,경과일수,제품수,냉동식품,기타,...,식료품,아침식사,통조림,"유제품,계란",가정용품,아기용품,간식,델리,누락,가격
0,2539329,1,prior,1,화요일,8,NaN,5.0,0,0,...,0,0,0,1,1,0,2,0,0,46.72
1,2398795,1,prior,2,수요일,7,15.0,6.0,0,0,...,0,1,0,0,0,0,3,0,0,49.17
2,473747,1,prior,3,수요일,12,21.0,5.0,0,0,...,1,0,0,1,0,0,2,0,0,59.18
3,2254736,1,prior,4,목요일,7,29.0,5.0,0,0,...,0,0,0,1,1,0,2,0,0,65.88
4,431534,1,prior,5,목요일,15,28.0,8.0,0,0,...,0,0,0,1,0,0,2,0,0,68.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3346078,2266710,206209,prior,10,금요일,18,29.0,9.0,1,0,...,1,1,0,1,1,0,1,0,0,46.11
3346079,1854736,206209,prior,11,목요일,10,30.0,8.0,0,0,...,0,0,0,2,1,0,1,1,0,48.43
3346080,626363,206209,prior,12,월요일,12,18.0,20.0,0,0,...,1,2,0,4,3,0,1,1,0,111.42
3346081,2977660,206209,prior,13,월요일,12,7.0,9.0,0,0,...,0,1,0,2,0,0,4,0,0,72.28


In [53]:
주문_고객.to_csv('../data/주문_고객.csv', index=False, encoding='euc-kr')

In [56]:
# 주문ID 별 가격의 합계를 계산
고객_가격 = 주문_고객.groupby('고객ID')['가격'].sum().reset_index()

# 결과 확인
고객_가격

,고객ID,가격
0,1,681.65
1,2,1647.29
2,3,488.82
3,4,205.68
4,5,278.56
...,...,...
206204,206205,372.68
206205,206206,2154.38
206206,206207,1591.25
206207,206208,4768.37


In [59]:
고객 = pd.read_csv('../data/고객.csv', encoding='euc-kr').drop(columns=['Unnamed: 0'])
고객

,고객ID,월요일빈도수,화요일빈도수,수요일빈도수,목요일빈도수,금요일빈도수,토요일빈도수,일요일빈도수,자정(0~3),새벽(3~6),...,식료품,아침식사,통조림,"유제품,계란",가정용품,아기용품,간식,델리,누락,군집
0,1,3,2,2,4,0,0,0,0,0,...,1,4,0,17,3,0,25,0,0,0
1,2,6,5,2,1,1,0,0,0,0,...,11,3,4,49,0,0,47,24,0,2
2,3,2,1,3,0,0,0,6,0,0,...,4,0,0,21,1,0,9,2,0,0
3,4,0,0,0,2,2,1,0,0,0,...,0,0,1,0,2,0,1,2,0,2
4,5,1,0,2,0,0,0,2,0,0,...,3,0,1,9,1,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206204,206205,1,1,0,1,1,0,0,0,0,...,1,0,0,20,0,1,0,5,2,0
206205,206206,8,7,11,12,3,8,18,0,0,...,14,1,9,39,12,0,42,5,0,1
206206,206207,3,3,2,1,2,2,3,0,1,...,15,4,11,52,0,1,23,8,0,3
206207,206208,10,12,7,5,5,5,5,0,1,...,38,17,11,161,6,3,59,28,2,1


In [60]:
# 고객과 고객_가격 데이터프레임을 고객ID를 기준으로 병합
고객 = pd.merge(고객, 고객_가격, on='고객ID', how='left')
고객

,고객ID,월요일빈도수,화요일빈도수,수요일빈도수,목요일빈도수,금요일빈도수,토요일빈도수,일요일빈도수,자정(0~3),새벽(3~6),...,아침식사,통조림,"유제품,계란",가정용품,아기용품,간식,델리,누락,군집,가격
0,1,3,2,2,4,0,0,0,0,0,...,4,0,17,3,0,25,0,0,0,681.65
1,2,6,5,2,1,1,0,0,0,0,...,3,4,49,0,0,47,24,0,2,1647.29
2,3,2,1,3,0,0,0,6,0,0,...,0,0,21,1,0,9,2,0,0,488.82
3,4,0,0,0,2,2,1,0,0,0,...,0,1,0,2,0,1,2,0,2,205.68
4,5,1,0,2,0,0,0,2,0,0,...,0,1,9,1,0,1,1,0,0,278.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206204,206205,1,1,0,1,1,0,0,0,0,...,0,0,20,0,1,0,5,2,0,372.68
206205,206206,8,7,11,12,3,8,18,0,0,...,1,9,39,12,0,42,5,0,1,2154.38
206206,206207,3,3,2,1,2,2,3,0,1,...,4,11,52,0,1,23,8,0,3,1591.25
206207,206208,10,12,7,5,5,5,5,0,1,...,17,11,161,6,3,59,28,2,1,4768.37


In [62]:
고객.to_csv('../data/고객.csv', index=False, encoding='euc-kr')